In [1]:
import trimesh

In [3]:
mesh = trimesh.load_mesh('./bunny.obj')

In [4]:
import igl

In [5]:
_,v_,f_,_,_ = igl.decimate(mesh.vertices,mesh.faces, 1000)

In [6]:
mesh = trimesh.Trimesh(v_,f_)

In [7]:
import numpy as np

In [8]:
normal = np.random.randn(3)
normal /= np.linalg.norm(normal)
point = np.random.randn(3)

In [24]:
!pip install triangle

In [54]:
bbox_mesh = trimesh.Trimesh()

In [11]:
def generate_box_mesh(v):
    v1 = 0
    v2 = 1
    v3 = 2
    v4 = 3
    v5 = 4
    v6 = 5
    v7 = 6
    v8 = 7

    bbox_tri = np.array([[v1, v2, v5], [v2, v5, v6], [v2, v3, v6], \
                    [v6, v7, v3], [v3, v7, v8], [v8, v4, v3], [v4, v8, v5],\
                   [v5, v4, v1], [v5, v6, v8], [v8, v6, v7], [v2, v4, v1], [v2, v4, v3]
                   ])

    zmin = v[:,-1].min() + 0.1 * v[:,-1].min()
    zmax = v[:,-1].max() + 0.1 * v[:,-1].max()
    ymin = v[:,-2].min() + 0.1 * v[:,-2].min() 
    ymax = v[:, -2].max() + 0.1 * v[:, -2].max() 
    xmin = v[:, -3].min() + 0.1 * v[:, -3].min() 
    xmax = v[:, -3].max() + v[:, -3].max()

    bbox_verts = np.array([[xmin, ymin, zmax],[xmax,ymin,zmax],[xmax, ymin, zmin],\
                  [xmin, ymin, zmin], [xmin, ymax, zmax], [xmax, ymax, zmax],\
                  [xmax, ymax, zmin], [xmin, ymax, zmin]])
    
    return bbox_verts, bbox_tri

In [98]:
bbox_verts, bbox_tri = generate_box_mesh(mesh.vertices)

bbox_mesh = trimesh.Trimesh(bbox_verts, bbox_tri)

In [99]:
from meshplot import plot

In [100]:

normal = np.random.randn(3)
normal /= np.linalg.norm(normal)
point = np.random.randn(3)

point = np.array([-0.54905429,  0.14446151, -0.16177154])
normal = np.array([-0.19714132, -0.87347182,  0.44517669])
polyline, triangles_edge = trimesh.intersections.mesh_plane(mesh, 
                                                           plane_origin= point, 
                                                           plane_normal= normal, 
                                                           return_faces=True)

# print(polyline.shape)
m = plot(mesh.vertices)
# m.add_mesh(bbox_verts, bbox_tri)
m.add_points(polyline.reshape(-1,3), c='blue')

/home/thor/anaconda3/envs/iglenv/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016663…

1

In [101]:
point,normal


(array([-0.54905429,  0.14446151, -0.16177154]),
 array([-0.19714132, -0.87347182,  0.44517669]))

In [102]:
bbox_mesh = trimesh.Trimesh()
bbox_mesh.vertices = bbox_verts
bbox_mesh.faces  = bbox_tri

In [103]:
def unordered_to_triangles(points):
    triangles = [[points[0], points[1], points[2]], [points[1], points[2],points[3]]]
    return np.array(triangles)

In [115]:
def split_mesh(mesh, plane_origin, plane_normal):
    """
    Splits mesh by intersection with a plane, 
    returns closed mesh for positive and negative side of plane 
    """
    plane_normal = (plane_normal)
    
    polyline, triangles_edge = trimesh.intersections.mesh_plane(mesh, 
                                                           plane_origin= plane_origin, 
                                                           plane_normal= plane_normal, 
                                                           return_faces=True)
    
    vertices_negative = []
    vertices_positive = []
    for vertex_id, vertex in enumerate(mesh.vertices):
        # TODO: add threshold
        signed_distance = np.dot(plane_normal, np.subtract(vertex, plane_origin))
        if signed_distance < 0:
            vertices_negative.append(vertex_id)
        else:
            vertices_positive.append(vertex_id)
    
    triangles_negative = []
    triangles_positive = []
    # For each triangle check how many points on either side of the plane
    for triangle in mesh.faces:
        triangle_negative_check = [vertex_id in vertices_negative for vertex_id in triangle]
        # If all points on the negative side this triangle is not affected
        if all(triangle_negative_check):
            triangles_negative.append(np.array(mesh.vertices[triangle]))
        elif any(triangle_negative_check):
            # Get section line for this triangle on edge
            section = polyline[np.all(np.isin(mesh.faces[triangles_edge], np.array(triangle)), axis=1)][0]
            assert section.shape == (2,3)
            # If two points on negative side
            if sum(triangle_negative_check) == 2:
                vertex_1, vertex_2 = mesh.vertices[triangle[triangle_negative_check]]
                
                triangles_negative.extend(unordered_to_triangles([section[0], section[1], vertex_1, vertex_2]))
                vertex_3 = mesh.vertices[triangle[np.logical_not(triangle_negative_check)]]
                triangles_positive.append(np.vstack([section[0], section[1], vertex_3]))
            elif sum(triangle_negative_check) == 1:
                vertex_3 = mesh.vertices[triangle[triangle_negative_check]]
                triangles_negative.append(np.vstack([section[0], section[1], vertex_3]))
                vertex_1, vertex_2 = mesh.vertices[triangle[np.logical_not(triangle_negative_check)]]
#                 print(section[0], section[1], vertex_1, vertex_2)
                triangles_positive.extend(unordered_to_triangles([section[0], section[1], vertex_1, vertex_2]))
        # If all points on the positive side this triangle is not affected
        else:
            triangles_positive.append(np.array(mesh.vertices[triangle]))
            
    # Get triangulation of the split surface
    vertices_splitsurface = []
    for a, b in polyline:
        for vertex in [a,b]:
            if tuple(vertex) not in vertices_splitsurface:
                vertices_splitsurface.append(tuple(vertex))

    triangles_splitsurface = unordered_to_triangles(vertices_splitsurface)
    
    # Add triangles of split surface to both the positive and negative side
    triangles_negative.extend(triangles_splitsurface)
    triangles_positive.extend(triangles_splitsurface)
    

    
    return np.array(triangles_positive), np.array(triangles_negative)

In [116]:
pos_mesh, neg_mesh = split_mesh(mesh, point, normal)

In [117]:
pos_mesh = np.array(pos_mesh)
neg_mesh = np.array(neg_mesh)

In [118]:
m = plot(pos_mesh.reshape(-1, 3),c='blue')
m.add_points(neg_mesh.reshape(-1,3))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016663…

1